In [20]:
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [58]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
def preprocess_text_nltk(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]

    return ' '.join(tokens)

In [78]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [80]:
file_path = "speech.txt"
speech = read_file(file_path)
cleaned_text=preprocess_text_nltk(speech)
cleaned_text

'world must made safe democraci peac must plant upon test foundat polit liberti selfish end serv desir conquest dominion seek indemn materi compens sacrific shall freeli make one champion right mankind shall satisfi right made secur faith freedom nation make fight without rancor without selfish object seek noth shall wish share free peopl shall feel confid conduct oper belliger without passion observ proud punctilio principl right fair play profess fight … easier us conduct belliger high spirit right fair act without animu enmiti toward peopl desir bring injuri disadvantag upon arm opposit irrespons govern thrown asid consider human right run amuck let say sincer friend german peopl shall desir noth much earli reestablish intim relat mutual advantag us—howev hard may time believ spoken heart born present govern bitter month friendship—exercis patienc forbear would otherwis imposs shall happili still opportun prove friendship daili attitud action toward million men women german birth na

In [121]:
def split_text_into_chunks(text, chunk_size=300, overlap=50):
    """Split text into chunks with overlap."""
    if overlap >= chunk_size:
        raise ValueError("Overlap should be smaller than chunk size.")

    words = word_tokenize(text)
    chunks = [ " ".join(words[i : i + chunk_size]) for i in range(0, len(words), chunk_size - overlap)]
    
    return chunks


In [122]:
chunks = split_text_into_chunks(cleaned_text)

In [112]:
from langchain_community.embeddings import OllamaEmbeddings
import numpy as np
embedding = OllamaEmbeddings(model="mxbai-embed-large")

In [113]:
document_embeddings = [embedding.embed_query(chunk) for chunk in chunks]
embeddings_np = np.array(document_embeddings, dtype=np.float32)

In [114]:
document_embeddings

[[0.2690128684043884,
  -0.33014294505119324,
  0.3865107297897339,
  0.6709243655204773,
  -0.4097580909729004,
  -0.02992120385169983,
  -0.15482479333877563,
  0.5953450202941895,
  -0.20704592764377594,
  0.14255280792713165,
  0.15424659848213196,
  -0.2807754874229431,
  0.07767175883054733,
  -0.24394500255584717,
  -0.8152968287467957,
  -0.19261543452739716,
  -0.5867557525634766,
  -0.22639213502407074,
  -0.33208590745925903,
  0.49775752425193787,
  -0.17684462666511536,
  -0.2848072052001953,
  -1.576554536819458,
  0.15579962730407715,
  -0.2485705316066742,
  0.6196181774139404,
  0.27076148986816406,
  0.2760619819164276,
  1.203507423400879,
  0.9793043732643127,
  -0.026796502992510796,
  -0.12811985611915588,
  -0.30336830019950867,
  -0.7155166864395142,
  -0.11936106532812119,
  -0.3766310513019562,
  1.4097697734832764,
  -0.9944978952407837,
  0.03880220651626587,
  -0.4536517262458801,
  -0.04659207537770271,
  -0.15904879570007324,
  0.6512764096260071,
  -0.43

In [86]:
!pip install faiss-cpu 

In [115]:
import faiss
import numpy as np

In [116]:
embedding_dim = 1024

# Initialize FAISS index
index = faiss.IndexFlatL2(embedding_dim)

In [117]:
embeddings_np.shape

(3, 1024)

In [118]:
index.add(embeddings_np)

In [ ]:
def search_faiss(query, index, top_k=3):
    """Search FAISS index for the most similar embeddings."""
    query_embedding = embedding.embed_query(query) # Embed query
    query_embedding_np = np.array(query_embedding, dtype=np.float32).reshape(1, -1)
    
    if query_embedding_np.shape[1] != embedding_dim:
        raise ValueError(f"Query embedding dimension mismatch: Expected {embedding_dim}, got {query_embedding_np.shape[1]}")

    D, I = index.search(query_embedding_np, top_k)  
    return I[0] 

In [120]:
query = "The speech is a call to action what."
result = search_faiss(query, index)

print("Closest matching chunks:")
for idx in result:
    if 0 <= idx < len(chunks): 
        print(f"- {chunks[idx]}")
    else:
        print(f"- Index {idx} is out of bounds.")


Closest matching chunks:
- gentlemen congress perform thu address may mani month fieri trial sacrific ahead us fear thing lead great peac peopl war terribl disastr war civil seem balanc right preciou peac shall fight thing alway carri nearest hearts—for democraci right submit author voic govern right liberti small nation univers dominion right concert free peopl shall bring peac safeti nation make world last free task dedic live fortun everyth everyth pride know day come america privileg spend blood might principl gave birth happi peac treasur god help
- world must made safe democraci peac must plant upon test foundat polit liberti selfish end serv desir conquest dominion seek indemn materi compens sacrific shall freeli make one champion right mankind shall satisfi right made secur faith freedom nation make fight without rancor without selfish object seek noth shall wish share free peopl shall feel confid conduct oper belliger without passion observ proud punctilio principl right fair 